In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras import preprocessing 
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
import tensorflow as tf 
from tensorflow.keras import models, layers
import pandas as pd

In [4]:
base_path='/kaggle/input/kermany2018/OCT2017 /train'

In [5]:
classes = ['CNV', 'DME','DRUSEN','NORMAL']

In [6]:
# define an image data generator
data_gen = preprocessing.image.ImageDataGenerator(
    # define the preprocessing function that should be applied to all images
    preprocessing_function=preprocess_input,
     #rotation_range=45,
     #width_shift_range=0.2,
     #height_shift_range=0.2,
     #horizontal_flip=True, 
     #vertical_flip=True,
    #zoom_range=0.2,
    # shear_range=0.2    
)

In [7]:
train_data_gen = data_gen.flow_from_directory(
        directory=base_path,
        class_mode="categorical",
        classes=classes,
        batch_size=30,
        target_size=(224, 224)
)

In [8]:
# load in all images at once
xtrain, ytrain = next(train_data_gen)
xtrain.shape, ytrain.shape


In [9]:
def plotImages(images_arr):
    fig, axes = plt.subplots(6, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [11]:
plotImages(xtrain)
print(ytrain)

<h1>Model </h1>

In [12]:
K.clear_session()
model = keras.Sequential()
model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))
model.add(Dense(4, activation = 'softmax'))
model.layers[0].trainable = False
model.summary()



In [13]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])              

# observe the validation loss and stop when it does not improve after 3 iterations
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=3)
model_training = model.fit(xtrain, ytrain, epochs=40, batch_size = 50, validation_split=0.3, callbacks=[early_stopping_monitor], verbose=1)


In [16]:
model.save('/kaggle/Output/model_ResNet50_OCT.h5')

In [20]:
model_training.history.keys()
pd.DataFrame(model_training.history)[['loss', 'val_loss']].plot()
plt.xlabel('epochs')
plt.ylabel('loss')

#
pd.DataFrame(model_training.history)[['val_accuracy', 'accuracy']].plot()
plt.xlabel('epochs')
plt.ylabel('accuracy')

In [22]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

In [23]:
ax =plt.figure(figsize = (10,10))
cm = confusion_matrix(y_true=y_test_copy, y_pred=np.argmax(pred, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

disp.plot()                                 # change the size of the matrix
plt.xticks(rotation=45)